In [ ]:
#Instalamos la libreria de STREAMLIT
#%pip install streamlit

In [ ]:
#Instalamos la libreria de plotly
#%pip install plotly

In [ ]:
#Instalamos npm en visual studio code

! npm install localtunnel

In [ ]:
%%writefile app.py
#Creamos el archivo de la APP en el interprete principal (Python)
#Se tiene que empezar con esta linea, no se puede poner un comentario antes u otra cosa ya que si no no se generará

#Importamos librerias
import streamlit as st
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
# Importar pathlib para manejar rutas de archivos
from pathlib import Path
import streamlit as st
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import io

# Función para cargar archivo Styless.css externo
def load_css():
    css_path = Path(__file__).parent / "styles.css"
    try:
        with open(css_path, "r", encoding="utf-8") as f:
            st.markdown(f"<style>{f.read()}</style>", unsafe_allow_html=True)
    except Exception as e:
        st.warning(f"No se pudo cargar CSS: {str(e)}")
#Carga css        
load_css()
##################################################################################################
#Definimos la instancia (función para cargar los datos)
@st.cache_resource
###################################################################################################    
#Creamos la función de carga de datos
def load_data():
    #Lectura del archivo csv
    df=pd.read_csv("venecia_limpio.csv")
    
    # Limpia datos de la columna 'price'
    df['price'] = df['price'].replace(r'[\$,]', '', regex=True).astype(float)
    ######################################################################################
    # Crear variables dicotómicas para la regresión logística
    #bathrooms_text
    df["bathroomsd"] = df["bathrooms_text"].astype(str).apply(lambda x: 1 if "2" in x or "3" in x or "4" in x else 0)
    
    #room_type Entire home
    df["E_room_typed"] = df["room_type"].apply(lambda x: 1 if x == "Entire home/apt" else 0)
    
    #room_type Private home
    df["P_room_typed"] = df["room_type"].apply(lambda x: 1 if x == "Private room" else 0)
    
    #room_type Hotel home
    df["H_room_typed"] = df["room_type"].apply(lambda x: 1 if x == "Hotel room" else 0)
    
    #instant_bookabled
    if "instant_bookable" in df.columns:
        df["instant_bookabled"] = df["instant_bookable"].apply(lambda x: 1 if x == "t" else 0)
    
    #Price
    Mprice = df["price"].median()
    df["D_price"] = df["price"].apply(lambda x: 1 if x >= Mprice else 0)
    
    #host_total_listings_count
    if "host_total_listings_count" in df.columns:
        Mlistings = df["host_total_listings_count"].median()
        df["D_host_listings"] = df["host_total_listings_count"].apply(lambda x: 1 if x >= Mlistings else 0)
    
    #review_scores_accuracy
    if "review_scores_accuracy" in df.columns:
        Maccuracy = df["review_scores_accuracy"].median()
        df["D_accuracy"] = df["review_scores_accuracy"].apply(lambda x: 1 if x >= Maccuracy else 0)
    
    #availability_30
    if "availability_30" in df.columns:
        Mavailability = df["availability_30"].median()
        df["D_availability"] = df["availability_30"].apply(lambda x: 1 if x >= Mavailability else 0)
    
    #availability_90
    if "availability_90" in df.columns:
        Mavailability90 = df["availability_90"].median()
        df["D_availability90"] = df["availability_90"].apply(lambda x: 1 if x >= Mavailability90 else 0)
    
    #number_of_reviews
    if "number_of_reviews" in df.columns:
        Mreviews = df["number_of_reviews"].median()
        df["D_reviews"] = df["number_of_reviews"].apply(lambda x: 1 if x >= Mreviews else 0)
    
    #review_scores_rating
    if "review_scores_rating" in df.columns:
        Mrating = df["review_scores_rating"].median()
        df["D_rating"] = df["review_scores_rating"].apply(lambda x: 1 if x >= Mrating else 0)
    
    #accommodates
    if "accommodates" in df.columns:
        Maccommodates = df["accommodates"].median()
        df["D_accommodates"] = df["accommodates"].apply(lambda x: 1 if x >= Maccommodates else 0)
    
    ######################################################################################
    #Separa columnas númericas y categóricas
    #Selecciono las columnas tipo numericas del dataframe
    numeric_df=df.select_dtypes(['float', 'int']) #Devuelve Columnas
    numeric_cols=numeric_df.columns.tolist()  #Devuelve lista de Columnas
    
    #Selecciono las columnas tipo texto del dataframe
    text_df=df.select_dtypes(['object']) #Devuelve Columnas
    text_cols= text_df.columns.tolist() #Devuelve lista de Columnas
    ##########
    ##########
    #Variables específicas mencionadas por el usuario
    specific_categorical_vars = [
        "host_response_rate", "host_acceptance_rate", "host_total_listings_count", 
        "accommodates", "bathrooms_text", "beds", "price", "maximum_nights_avg_ntm", 
        "availability_365", "number_of_reviews", "review_scores_location",  
        "review_scores_value", "reviews_per_month", "review_scores_communication", 
        "review_scores_checkin", "review_scores_cleanliness", "review_scores_accuracy"
    ]
    
    # Filtrar para solo incluir las que realmente están en el dataframe
    available_specific_cats = [col for col in specific_categorical_vars if col in df.columns]
    
    # Crear lista de variables dicotómicas
    dichotomous_cols = [col for col in df.columns if col.startswith('D_') or col.endswith('d')]
    ######
    ######
    
    #Selecciono algunas columnas categoricas de valores para desplegar en diferentes cuadros
    categorical_column_room= df['room_type']
    #Obtengo los valores unicos de la columna categórica seleccionada
    unique_categories_room= categorical_column_room.unique()

    return df, numeric_cols, text_cols, unique_categories_room, numeric_df, available_specific_cats, dichotomous_cols
################################################################################

################################################################################
#Cargo los datos obtenidos de la función "load_data"
df, numeric_cols, text_cols,unique_categories_room, numeric_df, specific_categorical_vars, dichotomous_cols=load_data()  
#######################################################################################
#Generamos los encabezados para la barra lateral (sibedar)
st.sidebar.title("DASHBOARD")
st.sidebar.subheader("Panel de Seleccion")

############################################################################### 
# Checkboxes para seleccionar las diferentes vistas
mostrar_portada= st.sidebar.checkbox("🏠 Portada", value=True)
mostrar_dataset= st.sidebar.checkbox("📂Mostrar Dataset Completo", value=False)
mostrar_univariado=st.sidebar.checkbox("🔍 Análisis Univariado", value=False)
mostrar_reg_simple=st.sidebar.checkbox("📈 Regresión Lineal Simple", value=False)
mostrar_reg_multiple=st.sidebar.checkbox("📉 Regresión Lineal Múltiple", value=False)
mostrar_reg_logistica = st.sidebar.checkbox("🧠 Regresión Logística", value=False)
#################################################################################
# Determinar qué vista mostrar (la primera que esté seleccionada)
if mostrar_portada:
    View = "Portada"
elif mostrar_dataset:
    View = "Mostrar Dataset Completo"
elif mostrar_univariado:
    View = "Análisis Univariado"
elif mostrar_reg_simple:
    View = "Regresión Lineal Simple"
elif mostrar_reg_multiple:
    View = "Regresión Lineal Múltiple"
elif mostrar_reg_logistica:
    View="Regresión Logística"   
else:
    View = "Portada"  # Vista por defecto si ninguna está seleccionada

# --- PORTADA ---
if View=="Portada":
    st.title("🌍 Dashboard Airbnb Venecia-Italia")
    st.subheader("Análisis de datos para la ciudad de Venecia")

    st.image("Venecia.jpg", caption="Venecia, Italia")

    st.markdown("""
    <p class="justified-text">Bienvenido/a a este análisis de los datos de hospedajes en Airbnb para la ciudad de Venecia.  
    En este Dashboard exploraremos las características principales de las propiedades, los tipos de alojamiento,
    su disponibilidad, y los precios más comunes, todo ello basado en datos reales.
    Este análisis tiene como objetivo conocer el comportamiento del mercado turístico en Venecia para posibles
    recomendaciones o visualizaciones interesantes.</p>
    """, unsafe_allow_html=True)
    
    # Información general del dataset
    st.subheader("Información General del Dataset")
    col1, col2, col3 = st.columns(3)
    
    with col1:
        st.metric("Total de Alojamientos", f"{len(df)}")
    
    with col2:
        st.metric("Precio Promedio", f"${df['price'].mean():.2f}")
    
    with col3:
        if 'room_type' in df.columns:
            most_common_type = df['room_type'].value_counts().idxmax()
            st.metric("Tipo de Habitación más Común", most_common_type)
    
    # Explorar el Dashboard
    st.subheader("Explorar el Dashboard")
    st.markdown("""
    Utilice el **Panel de Navegación** ubicado en la barra lateral para navegar entre las diferentes secciones del dashboard:
    
    - **📋 Dataset Completo**: Visualización y exploración del conjunto de datos completo.
    - **📊 Análisis Univariado**: Análisis de características individuales y distribuciones.
    - **📈 Regresión Lineal Simple**: Modelado predictivo utilizando una variable independiente.
    - **📊 Regresión Lineal Múltiple**: Modelado predictivo utilizando múltiples variables independientes.
    - **🔄 Regresión Logística**: Clasificación basada en características seleccionadas.
    """)
###############################################################################################################################
# CONTENIDO DE LA VISTA 2 DATASET COMPLETO   
elif View=="Mostrar Dataset Completo":
    st.title("📋 Dataset Completo de Airbnb Venecia")
    st.markdown("""
    <p class="justified-text">A continuación se muestra el conjunto de datos completo de Airbnb para la ciudad de Venecia.
    Explore los datos para obtener una visión general de los alojamientos disponibles.</p>
    """, unsafe_allow_html=True)
    
    # Mostrar el dataset completo
    st.write(df)
    
    st.write(df.columns)
    # Mostrar resumen estadístico
    st.subheader("Resumen Estadístico")
    st.write(df.describe())
    
#############################################################################################################################
############################################################################################################
# CONTENIDO DE LA VISTA 3: ANÁLISIS UNIVARIADO
elif View == "Análisis Univariado":
    st.title("📊 Análisis Univariado - Extracción de Características")
    st.markdown("""
    <p class="justified-text">El análisis univariado examina la distribución de cada variable individual, 
    permitiéndonos comprender las características básicas de los datos.
    Aquí analizamos las variables categóricas más significativas de los alojamientos Airbnb en Venecia.</p>
    """, unsafe_allow_html=True)
    
    # Widget 1: Checkbox
    # Generar 3 checkbox para mostrar diferentes elementos
    col1, col2, col3 = st.columns(3)
    with col1:
        check_dataset = st.checkbox("📋 Mostrar Dataset", key="check_dataset")
    with col2:
        check_summary = st.checkbox("📑 Mostrar Resumen Estadístico", key="check_summary")
    with col3:
        check_col_info = st.checkbox("ℹ️ Mostrar Info de Columnas", key="check_col_info")
    
    # Mostrar dataset según selección
    if check_dataset:
        st.subheader("Dataset de Airbnb Venecia")
        st.dataframe(df.head(10))
    
    if check_summary:
        st.subheader("Resumen Estadístico")
        st.dataframe(df.describe())
    
    if check_col_info:
        st.subheader("Información de Columnas")
        st.write("**Variables Numéricas:**", numeric_cols)
        st.write("**Variables Categóricas:**", text_cols)
    
    # Widget 2: MultiSelect para variables categóricas
    st.sidebar.subheader("Análisis Categórico")
    categorical_cols = df.select_dtypes(include='object').columns.tolist()
    selected_cat_columns = st.sidebar.multiselect(
        "Selecciona Variables Categóricas para Analizar:",
        options=categorical_cols,
        default=categorical_cols[:2] if len(categorical_cols) >= 2 else categorical_cols
    )
    
    # Visualizaciones para variables categóricas
    if selected_cat_columns:
        st.subheader("Análisis de Variables Categóricas")
        
        # Widget 3: Selectbox para tipo de gráfico
        chart_type = st.sidebar.selectbox(
            "Tipo de Gráfico:",
            ["Gráfico Circular", "Gráfico de Barras", "Tabla de Frecuencias"],
            key="chart_type_cat"
        )
        
        for col in selected_cat_columns:
            st.markdown(f"### Distribución de {col}")
            
            # Calcular y mostrar frecuencias
            value_counts = df[col].value_counts().reset_index()
            value_counts.columns = [col, 'Frecuencia']
            value_counts['Porcentaje'] = (value_counts['Frecuencia'] / value_counts['Frecuencia'].sum() * 100).round(2)
            
            if chart_type == "Gráfico Circular":
                fig = px.pie(
                    value_counts, 
                    names=col, 
                    values='Frecuencia',
                    title=f"Distribución de {col}",
                    hole=0.4,
                    color_discrete_sequence=px.colors.qualitative.Pastel
                )
                st.plotly_chart(fig, use_container_width=True)
            
            elif chart_type == "Gráfico de Barras":
                fig = px.bar(
                    value_counts,
                    x=col,
                    y='Frecuencia',
                    title=f"Distribución de {col}",
                    color=col,
                    text='Frecuencia'
                )
                fig.update_traces(texttemplate='%{text}', textposition='outside')
                fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
                st.plotly_chart(fig, use_container_width=True)
            
            elif chart_type == "Tabla de Frecuencias":
                st.dataframe(value_counts, use_container_width=True)
                
# Widget 4: Button para análisis adicional
st.sidebar.subheader("Análisis Adicional")
if st.sidebar.button("📈 Analizar Tendencias", key="btn_trend"):
    st.subheader("Tendencias por Tipo de Habitación")
            
    # Análisis de precio promedio por tipo de habitación
    avg_price = df.groupby('room_type')['price'].mean().reset_index()
            
    fig = px.bar(
        avg_price,
        x='room_type',
        y='price',
        title="Precio Promedio por Tipo de Habitación",
        labels={'price': 'Precio Promedio', 'room_type': 'Tipo de Habitación'},
        color='room_type'
    )
    st.plotly_chart(fig, use_container_width=True)
#######################################################################################################
#CONTENIDO DE LA VISTA 4: REGRESIÓN LINEAL SIMPLE
elif View == "Regresión Lineal Simple":
    st.title("📈 Modelado Predictivo - Regresión Lineal Simple")
    st.markdown("""
    <p class="justified-text">La regresión lineal simple nos permite modelar la relación entre dos variables cuantitativas,
    utilizando una variable para predecir el valor de la otra. En este caso, analizaremos qué factores influyen
    en el precio de los alojamientos Airbnb en Venecia.</p>
    """, unsafe_allow_html=True)
    
    # Widget 1: Selectbox para variables X e Y
    st.sidebar.subheader("Variables del Modelo")
    
    target_var = st.sidebar.selectbox(
        "Variable a Predecir (Y):",
        options=numeric_cols,
        index=numeric_cols.index('price') if 'price' in numeric_cols else 0
    )
    
    feature_var = st.sidebar.selectbox(
        "Variable Predictora (X):",
        options=[col for col in numeric_cols if col != target_var],
        index=0
    )
    
    # Widget 2: Checkbox para mostrar detalles del modelo
    show_details = st.sidebar.checkbox("📋 Mostrar Detalles del Modelo", value=True)
    
    # Widget para mostrar mapa de calor
    show_heatmap = st.sidebar.checkbox("🔥 Mostrar Mapa de Calor", value=False)

    # Creación del modelo de regresión lineal simple
    X = df[[feature_var]].values
    y = df[target_var].values
    
    # División en conjunto de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Entrenamiento del modelo
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Predicciones
    y_pred = model.predict(X_test)
    
    # Evaluación del modelo
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Visualización del modelo
    st.subheader(f"Modelo de Regresión Lineal: {feature_var} → {target_var}")
    
    # Crear gráfico de dispersión con línea de regresión
    fig = px.scatter(
        x=X_test.flatten(), 
        y=y_test,
        labels={
            'x': feature_var,
            'y': target_var
        },
        title=f"Regresión Lineal Simple: {feature_var} vs {target_var}"
    )
    
    # Añadir línea de regresión
    x_range = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)
    y_range = model.predict(x_range)
    
    fig.add_trace(
        go.Scatter(
            x=x_range.flatten(),
            y=y_range,
            mode='lines',
            name='Línea de Regresión',
            line=dict(color='red', width=2)
        )
    )
    
    st.plotly_chart(fig, use_container_width=True)
    
    # Mostrar métricas en dos columnas
    col1, col2 = st.columns(2)
    
    with col1:
        st.metric(
            label="Coeficiente R²",
            value=f"{r2:.4f}",
            delta="Bueno" if r2 > 0.7 else "Moderado" if r2 > 0.5 else "Bajo"
        )
    
    # Ecuación de la recta
    slope = model.coef_[0]
    intercept = model.intercept_
    
    # Mostrar mapa de calor si se seleccionó
    if show_heatmap:
        st.subheader("Mapa de Calor de Correlaciones")
        
        # Seleccionar variables relevantes para el mapa de calor
        heatmap_vars = [target_var, feature_var]
        # Añadir algunas variables más para enriquecer el mapa de calor
        additional_vars = [col for col in numeric_cols if col not in heatmap_vars][:5]  # Limitamos a 5 variables adicionales
        heatmap_vars.extend(additional_vars)
        
        # Calcular matriz de correlación
        heatmap_df = df[heatmap_vars]
        corr_matrix = heatmap_df.corr()
        
        # Crear mapa de calor con plotly
        fig_heatmap = px.imshow(
            corr_matrix,
            text_auto=True,
            aspect="auto",
            color_continuous_scale='RdBu_r',
            title="Correlaciones entre Variables Seleccionadas"
        )
        fig_heatmap.update_layout(height=500)
        st.plotly_chart(fig_heatmap, use_container_width=True)
        
        # Explicación del mapa de calor
        st.info("""
        **Interpretación del mapa de calor:**
        - Valores cercanos a 1 (azul oscuro): Correlación positiva fuerte
        - Valores cercanos a -1 (rojo oscuro): Correlación negativa fuerte
        - Valores cercanos a 0 (colores claros): Poca o ninguna correlación
        
        Las variables altamente correlacionadas entre sí pueden proporcionar información similar al modelo.
        """)
    
    # Detalles del modelo
    if show_details:
        st.subheader("Detalles del Modelo")
        
        # Explicación de los coeficientes
        st.write(f"""
        - **Coeficiente de determinación (R²):** {r2:.4f}
          - Interpretación: El modelo explica el {r2*100:.2f}% de la variabilidad en {target_var}.
        """)
########################################################################################################################
#CONTENIDO DE LA VISTA 5: MODELADO PREDICTIVO - REGRESIÓN LINEAL MÚLTIPLE
elif View == "Regresión Lineal Múltiple":
    st.title("📊 Modelado Predictivo - Regresión Lineal Múltiple")
    st.markdown("""
    <p class="justified-text">La regresión lineal múltiple nos permite modelar la relación entre varias variables predictoras
    y una variable dependiente. Este enfoque es útil para identificar qué combinación de factores influye más
    en el precio de los alojamientos Airbnb en Venecia.</p>
    """, unsafe_allow_html=True)
    
    # Widget 1: Selectbox para variable objetivo
    st.sidebar.subheader("Variables del Modelo")
    
    target_var = st.sidebar.selectbox(
        "Variable a Predecir:",
        options=numeric_cols,
        index=numeric_cols.index('price') if 'price' in numeric_cols else 0,
        key="target_multi"
    )
    
    # Widget 2: Multiselect para variables predictoras
    predictor_vars = st.sidebar.multiselect(
        "Variables Predictoras:",
        options=[col for col in numeric_cols if col != target_var],
        default=[col for col in numeric_cols if col != target_var][:3]  # Seleccionar primeras 3 por defecto
    )
    
    # Widget para mostrar matriz de correlación
    show_corr_matrix = st.sidebar.checkbox("🔍 Mostrar Matriz de Correlación", value=True)
    
    # Verificar si hay suficientes variables seleccionadas
    if len(predictor_vars) < 2:
        st.warning("Por favor, selecciona al menos 2 variables predictoras para la regresión múltiple.")
    else:
        # Mostrar matriz de correlación si se seleccionó
        if show_corr_matrix:
            st.subheader("Matriz de Correlación")
            
            # Incluir variable objetivo y predictoras
            corr_vars = predictor_vars + [target_var]
            corr_df = df[corr_vars]
            
            # Calcular y mostrar matriz de correlación
            corr_matrix = corr_df.corr()
            
            # Crear mapa de calor con plotly
            fig_corr = px.imshow(
                corr_matrix,
                text_auto=True,
                aspect="auto",
                color_continuous_scale='RdBu_r',
                title="Correlaciones entre Variables del Modelo"
            )
            fig_corr.update_layout(height=500)
            st.plotly_chart(fig_corr, use_container_width=True)
            
            # Advertencia sobre multicolinealidad
            high_corr = False
            for i in range(len(predictor_vars)):
                for j in range(i+1, len(predictor_vars)):
                    if abs(corr_matrix.loc[predictor_vars[i], predictor_vars[j]]) > 0.7:
                        high_corr = True
                        break
            
            if high_corr:
                st.warning("""
                ⚠️ **Advertencia:** Se detectó alta correlación entre algunas variables predictoras.
                Esto puede causar problemas de multicolinealidad en el modelo. Considera eliminar
                algunas de las variables altamente correlacionadas.
                """)
        
        # Procesamiento del modelo
        X = df[predictor_vars].values
        y = df[target_var].values
        
        # División en conjunto de entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Entrenamiento del modelo
        model_multi = LinearRegression()
        model_multi.fit(X_train, y_train)
        
        # Predicciones
        y_pred_multi = model_multi.predict(X_test)
        
        # Evaluación del modelo
        mse_multi = mean_squared_error(y_test, y_pred_multi)
        r2_multi = r2_score(y_test, y_pred_multi)
        
        # Visualización del modelo
        st.subheader("Resultados del Modelo de Regresión Múltiple")
        
        # Mostrar métricas en dos columnas
        col1, col2 = st.columns(2)
        
        with col1:
            st.metric(
                label="Coeficiente R²",
                value=f"{r2_multi:.4f}",
                delta="Bueno" if r2_multi > 0.7 else "Moderado" if r2_multi > 0.5 else "Bajo"
            )
        # Mostrar coeficientes
        st.subheader("Coeficientes del Modelo")
        
        coef_df = pd.DataFrame({
            'Variable': predictor_vars,
            'Coeficiente': model_multi.coef_
        })
        coef_df['Valor Absoluto'] = abs(coef_df['Coeficiente'])
        coef_df = coef_df.sort_values(by='Valor Absoluto', ascending=False)
        
        # Visualización de coeficientes
        fig_coef = px.bar(
            coef_df,
            x='Variable',
            y='Coeficiente',
            title="Importancia de Variables en el Modelo",
            color='Valor Absoluto',
            text='Coeficiente'
        )
        fig_coef.update_traces(texttemplate='%{text:.4f}', textposition='outside')
        st.plotly_chart(fig_coef, use_container_width=True)
        
        # Ecuación del modelo
        intercept_multi = model_multi.intercept_
        
        equation = f"{target_var} = {intercept_multi:.4f}"
        for var, coef in zip(predictor_vars, model_multi.coef_):
            sign = "+" if coef >= 0 else ""
            equation += f" {sign} {coef:.4f} × {var}"
################################################################################################################
# CONTENIDO DE LA VISTA 6: MODELADO PREDICTIVO - REGRESIÓN LOGÍSTICA
elif View == "Regresión Logística":
    st.title("🧠 Modelado Predictivo - Regresión Logística")
    st.markdown("""
    <p class="justified-text">La regresión logística es un método estadístico para predecir una variable categórica binaria
    a partir de otras variables predictoras. Este enfoque es útil para predecir categorías en alojamientos Airbnb
    basándose en características numéricas.</p>
    """, unsafe_allow_html=True)
    
    # Widget 1: Selectbox para variable objetivo categórica
    st.sidebar.subheader("Variables del Modelo")
    
    # Convertimos la variable de precio a binaria para la demostración de regresión logística
    if 'price_category' not in df.columns:
        # Calcular el precio medio
        mean_price = df['price'].mean()
        # Crear variable binaria: 1 si precio > precio medio, 0 en caso contrario
        df['price_category'] = (df['price'] > mean_price).astype(int)
        # Añadir etiquetas descriptivas
        df['price_label'] = df['price_category'].map({1: 'Precio Alto', 0: 'Precio Bajo'})
    
    # Si hay otras variables categóricas, las podemos usar también
    binary_cols = []
    for col in df.columns:
        if df[col].dtype == 'object' and df[col].nunique() <= 5:
            binary_cols.append(col)
    
    # Agregamos nuestra variable de precio convertida
    categorical_targets = ['price_category'] + binary_cols
    
    target_cat = st.sidebar.selectbox(
        "Variable Categórica a Predecir:",
        options=categorical_targets,
        index=0,
        key="target_logistic"
    )
    
    # Si la variable seleccionada no es binaria, mostramos un mensaje
    if target_cat != 'price_category' and df[target_cat].nunique() > 2:
        st.warning(f"""
        La variable seleccionada '{target_cat}' tiene {df[target_cat].nunique()} categorías.
        La regresión logística binaria requiere una variable objetivo con exactamente 2 categorías.
        Para esta demostración, usaremos 'price_category' como variable objetivo.
        """)
        target_cat = 'price_category'
    
    # Widget 2: Multiselect para variables predictoras
    predictor_vars_log = st.sidebar.multiselect(
        "Variables Predictoras:",
        options=[col for col in numeric_cols if col != 'price' and col != target_cat],
        default=[col for col in numeric_cols if col != 'price' and col != target_cat][:3]  # Seleccionar primeras 3 por defecto
    )
    
    if len(predictor_vars_log) < 1:
        st.warning("Por favor, selecciona al menos 1 variable predictora para la regresión logística.")
    else:
        # Preparar datos
        X_log = df[predictor_vars_log].values
        y_log = df[target_cat].values
        
        # División en conjunto de entrenamiento y prueba
        X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(
            X_log, y_log, test_size=0.2, random_state=42
        )
        
        # Creación del modelo de regresión logística
        model_log = LogisticRegression(max_iter=1000)
        model_log.fit(X_train_log, y_train_log)
        
        # Predicciones
        y_pred_log = model_log.predict(X_test_log)
        y_pred_proba_log = model_log.predict_proba(X_test_log)[:, 1]
        
        # Evaluación del modelo
        accuracy = accuracy_score(y_test_log, y_pred_log)
        precision = precision_score(y_test_log, y_pred_log)
        recall = recall_score(y_test_log, y_pred_log)
        f1 = f1_score(y_test_log, y_pred_log)
        
        # Visualización del modelo
        st.subheader("Resultados del Modelo de Regresión Logística")
        
        # Mostrar métricas en dos filas con dos columnas cada una
        col1, col2 = st.columns(2)
        with col1:
            st.metric(
                label="Exactitud (Accuracy)",
                value=f"{accuracy:.4f}",
                delta="Bueno" if accuracy > 0.7 else "Moderado" if accuracy > 0.5 else "Bajo"
            )
        with col2:
            st.metric(
                label="Precisión (Precision)",
                value=f"{precision:.4f}"
            )
        
        col3, col4 = st.columns(2)
        with col3:
            st.metric(
                label="Sensibilidad (Recall)",
                value=f"{recall:.4f}"
            )
        
        # Matriz de confusión
        st.subheader("Matriz de Confusión")
        conf_matrix = confusion_matrix(y_test_log, y_pred_log)
        
        # Reordenar la matriz de confusión según la imagen proporcionada
        # La imagen muestra: [[VP, FP], [FN, VN]]
        # Por defecto, confusion_matrix retorna: [[VN, FP], [FN, VP]]
        # Necesitamos reorganizar los elementos
        reordered_conf_matrix = np.array([
            [conf_matrix[1, 1], conf_matrix[0, 1]],  # [VP, FP]
            [conf_matrix[1, 0], conf_matrix[0, 0]]   # [FN, VN]
        ])
        
        # Crear matriz de confusión visualmente
        fig_conf = px.imshow(
            reordered_conf_matrix,
            text_auto=True,
            x=['Predicho Positivo', 'Predicho Negativo'],
            y=['Real Positivo', 'Real Negativo'],
            color_continuous_scale='Blues',
            title="Matriz de Confusión"
        )
        st.plotly_chart(fig_conf, use_container_width=True)
        
        # Extraer valores de la matriz de confusión para la interpretación
        VP = reordered_conf_matrix[0, 0]
        FP = reordered_conf_matrix[0, 1]
        FN = reordered_conf_matrix[1, 0]
        VN = reordered_conf_matrix[1, 1]
        
        # Calcular métricas adicionales como en la imagen
        exactitud = (VP + VN) / (VP + VN + FP + FN)
        precision = VP / (VP + FP) if (VP + FP) > 0 else 0
        sensibilidad = VP / (VP + FN) if (VP + FN) > 0 else 0
        especificidad = VN / (VN + FP) if (VN + FP) > 0 else 0
        tasa_falsos_negativos = FN / (FN + VP) if (FN + VP) > 0 else 0
        valor_predictivo_positivo = VP / (VP + FP) if (VP + FP) > 0 else 0
        valor_predictivo_negativo = VN / (VN + FN) if (VN + FN) > 0 else 0
        
        st.info("""
        **Interpretación de la matriz de confusión:**
        - **Verdaderos Positivos (VP - arriba izquierda):** Casos positivos correctamente predichos
        - **Falsos Positivos (FP - arriba derecha):** Casos negativos incorrectamente predichos como positivos (Error tipo 1)
        - **Falsos Negativos (FN - abajo izquierda):** Casos positivos incorrectamente predichos como negativos (Error tipo 2)
        - **Verdaderos Negativos (VN - abajo derecha):** Casos negativos correctamente predichos
        """)
        
       
        
        # Crear DataFrame con coeficientes
        coef_log_df = pd.DataFrame({
            'Variable': predictor_vars_log,
            'Coeficiente': model_log.coef_[0],
        })
        coef_log_df['Valor Absoluto'] = abs(coef_log_df['Coeficiente'])
        coef_log_df = coef_log_df.sort_values(by='Valor Absoluto', ascending=False)
        
       
        
        # Tabla de coeficientes
        st.subheader("Coeficientes")
        st.dataframe(coef_log_df[['Variable', 'Coeficiente']])
        
        st.info("""
        **Interpretación:**
        - **Coeficiente:** Indica el cambio en el logaritmo de las probabilidades por unidad de cambio en la variable predictora.
       """)

Overwriting app.py
